In [1]:
from main import client, Part, analysis_prompt, recover_json, save_incident, db

gcs_uri = "gs://keyanvisionaibucket/visionintelligence/panic_test.mp4"

In [6]:
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=[
        Part.from_uri(
            file_uri=gcs_uri,
            mime_type="video/mp4",
        ),
        analysis_prompt,
    ],
)

analysis_result = recover_json(response.text)

In [7]:
zone_id = "west-zone"
video_id = "123"
ts = "2025-07-25T00:00:00Z"

In [8]:
analysis_result

{'incidents': {'congestion': {'score': 1.0,
   'explanation': 'The video shows extreme congestion with people packed shoulder to shoulder, making free movement impossible and causing crushing pressure.',
   'timestamps': [{'start': 0.0, 'end': 0.17}]},
  'crowd_surges': {'score': 1.0,
   'explanation': 'Clear evidence of crowd surges is visible, particularly around 0:03-0:05 and 0:08-0:10, where individuals are seen being pushed, pulled, and losing their footing due to uncontrolled crowd movement.',
   'timestamps': [{'start': 0.0, 'end': 0.17}]},
  'unruly_behavior': {'score': 0.2,
   'explanation': "While the crowd is agitated and desperate to move, there isn't overt aggressive or unruly behavior directed at others. The pushing appears to be a direct consequence of the extreme congestion and surges, rather than intentional aggression.",
   'timestamps': []},
  'panic_or_danger': {'score': 1.0,
   'explanation': 'The video explicitly shows people in distress, some screaming, being cru

In [5]:
1/0.1

10.0

In [ ]:
incident_ids = []
if 'incidents' in analysis_result:
    for incident_type, incident_data in analysis_result['incidents'].items():
        if incident_data.get('score', 0) > 0.5:
            incident_id = save_incident(incident_type, incident_data, zone_id, video_id, ts)
            incident_ids.append(incident_id)

In [ ]:
# Save the overall analysis report
report_ref = db.collection('analysis_reports').document()
report_ref.set({
    "video_id": video_id,
    "video_uri": gcs_uri,
    "zone_id": zone_id,
    "created_at": ts,
    "overall_safety_assessment": analysis_result.get("overall_safety_assessment"),
    "recommended_actions": analysis_result.get("recommended_actions"),
    "incident_refs": [db.collection('incidents').document(id) for id in incident_ids]
})

print(f"Video analysis successful. Report ID: {report_ref.id}")
